In [5]:
import os
import time
import numpy as np
import pandas as pd

from sklearn.preprocessing import MultiLabelBinarizer, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score,
    balanced_accuracy_score,
    precision_score,
    recall_score,
    f1_score,
)

In [1]:
# 1. Load data
train = pd.read_csv("../gen/data_def_train_folds.csv")
test  = pd.read_csv("../gen/data_def_test.csv")

print("Train shape:", train.shape)
print("Test shape:", test.shape)

# 2. Create combined column for Phase 2 (intrinsic + extrinsic)
train["labels_1_2_intr_extr"] = (
    train[["labels_1_intrinsic", "labels_2_extrinsic"]]
    .fillna("")
    .agg("|".join, axis=1)
    .str.strip("|")
)

test["labels_1_2_intr_extr"] = (
    test[["labels_1_intrinsic", "labels_2_extrinsic"]]
    .fillna("")
    .agg("|".join, axis=1)
    .str.strip("|")
)

# 3. Target – NOVA group
target = "nova_group"

le = LabelEncoder()
y_train = le.fit_transform(train[target])
y_test = le.transform(test[target])

print("Class mapping:", dict(zip(le.classes_, le.transform(le.classes_))))

# 4. Define phases (name -> column)
phases = {
    "phase1_intrinsic": "labels_1_intrinsic",
    "phase2_intr+extr": "labels_1_2_intr_extr",
    "phase3_all_labels": "labels_string",
}

# 5. Best Random Forest hyperparameters per phase (from your NOVA CV)
best_params_by_phase = {
    "phase1_intrinsic": {
        "class_weight": "balanced",
        "max_depth": None,
        "max_features": "sqrt",
        "min_samples_leaf": 1,
        "min_samples_split": 10,
        "n_estimators": 50,
    },
    "phase2_intr+extr": {
        "class_weight": "balanced",
        "max_depth": None,
        "max_features": "log2",
        "min_samples_leaf": 1,
        "min_samples_split": 10,
        "n_estimators": 100,
    },
    "phase3_all_labels": {
        "class_weight": "balanced",
        "max_depth": None,
        "max_features": "log2",
        "min_samples_leaf": 1,
        "min_samples_split": 10,
        "n_estimators": 100,
    },
}

base_rf_params = {
    "random_state": 42,
    "n_jobs": -1,
}

# 6. Containers for results
all_metrics = []
coverage_stats = [] 

# 7. Loop over phases – fit/evaluate only
for phase_name, label_col in phases.items():
    print("\n" + "="*90)
    print(f"=== {phase_name} – Using column '{label_col}' ===")
    print("="*90)

    # 7.1 Encode label features for this phase
    train[label_col] = train[label_col].fillna("")
    test[label_col]  = test[label_col].fillna("")

    train_labels = train[label_col].apply(lambda x: x.split("|") if x != "" else [])
    test_labels  = test[label_col].apply(lambda x: x.split("|") if x != "" else [])

    mlb = MultiLabelBinarizer(sparse_output=True)
    X_train_labels = mlb.fit_transform(train_labels)
    X_test_labels  = mlb.transform(test_labels)

    print(f"Encoded label matrix shape (train): {X_train_labels.shape}")
    print(f"Encoded label matrix shape (test):  {X_test_labels.shape}")

    # ---- PhaseCoverage masks (at least 1 label in this phase)
    train_nonempty_mask = np.array(X_train_labels.sum(axis=1)).ravel() > 0
    test_nonempty_mask  = np.array(X_test_labels.sum(axis=1)).ravel() > 0

    # Basic counts for coverage
    train_total = len(y_train)
    test_total  = len(y_test)
    core_mask   = test["core_slice"].astype(bool).values
    core_total  = int(core_mask.sum())

    train_cov_n = int(train_nonempty_mask.sum())
    test_cov_n  = int(test_nonempty_mask.sum())
    core_cov_n  = int((test_nonempty_mask & core_mask).sum())

    # Store coverage stats
    coverage_stats.append({
        "Phase": phase_name,
        "Dataset": "Train",
        "n_total": train_total,
        "n_with_labels": train_cov_n,
        "frac_with_labels": train_cov_n / train_total if train_total > 0 else np.nan,
    })
    coverage_stats.append({
        "Phase": phase_name,
        "Dataset": "Test",
        "n_total": test_total,
        "n_with_labels": test_cov_n,
        "frac_with_labels": test_cov_n / test_total if test_total > 0 else np.nan,
    })
    coverage_stats.append({
        "Phase": phase_name,
        "Dataset": "Test-Core",
        "n_total": core_total,
        "n_with_labels": core_cov_n,
        "frac_with_labels": core_cov_n / core_total if core_total > 0 else np.nan,
    })

    print(
        f"Coverage – {phase_name}:\n"
        f"  Train:     {train_cov_n}/{train_total} "
        f"({train_cov_n/train_total:.3%} with ≥1 label in this phase)\n"
        f"  Test:      {test_cov_n}/{test_total} "
        f"({test_cov_n/test_total:.3%} with ≥1 label in this phase)\n"
        f"  Test-Core: {core_cov_n}/{core_total} "
        f"({core_cov_n/core_total:.3%} with ≥1 label in this phase)"
    )

    # 7.2 Fit Random Forest with best params for this phase
    rf_params = {**base_rf_params, **best_params_by_phase[phase_name]}
    rf = RandomForestClassifier(**rf_params)

    fit_start = time.perf_counter()
    rf.fit(X_train_labels, y_train)  # train on ALL rows
    fit_end = time.perf_counter()
    fit_time = fit_end - fit_start

    print(f"Fitted Random Forest for {phase_name} in {fit_time:.2f} s")

    # 7.3 Predictions: Train–Overall, Test–Overall, Test–Core
    # Train–Overall
    y_pred_train = rf.predict(X_train_labels)

    # Test–Overall
    y_pred_test = rf.predict(X_test_labels)

    # Test–Core subset
    y_test_core = y_test[core_mask]
    y_pred_test_core = y_pred_test[core_mask]

    # PhaseCoverage subsets (at least 1 label in this phase)
    y_train_cov = y_train[train_nonempty_mask]
    y_pred_train_cov = y_pred_train[train_nonempty_mask]

    y_test_cov = y_test[test_nonempty_mask]
    y_pred_test_cov = y_pred_test[test_nonempty_mask]

    # 7.4 Attach predictions back to DataFrames (model-specific cols)
    train[f"{phase_name}_rf_pred_enc"] = y_pred_train
    test[f"{phase_name}_rf_pred_enc"] = y_pred_test

    train[f"{phase_name}_rf_pred"] = le.inverse_transform(y_pred_train)
    test[f"{phase_name}_rf_pred"] = le.inverse_transform(y_pred_test)

    # 7.5 Compute metrics helper
    def compute_metrics(y_true, y_pred, phase, dataset_name):
        if len(y_true) == 0:
            return {
                "Phase": phase,
                "Dataset": dataset_name,
                "n_samples": 0,
                "Accuracy": np.nan,
                "Balanced Accuracy": np.nan,
                "Precision (Macro)": np.nan,
                "Recall (Macro)": np.nan,
                "F1 (Macro)": np.nan,
                "F1 (Micro)": np.nan,
            }

        return {
            "Phase": phase,
            "Dataset": dataset_name,
            "n_samples": len(y_true),
            "Accuracy": accuracy_score(y_true, y_pred),
            "Balanced Accuracy": balanced_accuracy_score(y_true, y_pred),
            "Precision (Macro)": precision_score(y_true, y_pred, average="macro", zero_division=0),
            "Recall (Macro)": recall_score(y_true, y_pred, average="macro", zero_division=0),
            "F1 (Macro)": f1_score(y_true, y_pred, average="macro", zero_division=0),
            "F1 (Micro)": f1_score(y_true, y_pred, average="micro", zero_division=0),
        }

    # Train metrics
    all_metrics.append(compute_metrics(y_train, y_pred_train, phase_name, "Train-Overall"))
    all_metrics.append(compute_metrics(y_train_cov, y_pred_train_cov, phase_name, "Train-PhaseCoverage"))
    # Test metrics
    all_metrics.append(compute_metrics(y_test, y_pred_test, phase_name, "Test-Overall"))
    all_metrics.append(compute_metrics(y_test_cov,  y_pred_test_cov,  phase_name, "Test-PhaseCoverage"))
    all_metrics.append(compute_metrics(y_test_core, y_pred_test_core, phase_name, "Test-Core"))

    # Print quick summary for this phase (last 5 entries)
    print("\nMetrics –", phase_name)
    for m in all_metrics[-5:]:
        print(
            f"{m['Dataset']}: n={m['n_samples']}, "
            f"Acc={m['Accuracy']:.4f}, "
            f"BalAcc={m['Balanced Accuracy']:.4f}, "
            f"F1_macro={m['F1 (Macro)']:.4f}"
        )

# 8. Combine and inspect metrics across phases & datasets
metrics_df = pd.DataFrame(all_metrics)
coverage_df = pd.DataFrame(coverage_stats)

print("\n=== All Metrics (Train-Overall, Test-Overall, Test-Core, PhaseCoverage) ===")
print(metrics_df)

print("\n=== Coverage stats per phase & dataset ===")
print(coverage_df)

# 9. Save outputs
metrics_df.to_csv("../results/rf_nova_eval_metrics_train_test_core.csv", sep=";", index=False)
coverage_df.to_csv("../results/rf_nova_phase_coverage_stats.csv", sep=";", index=False)
train.to_csv("../results/data_def_train_with_rf_nova_preds.csv", sep=";", index=False)
test.to_csv("../results/data_def_test_with_rf_nova_preds.csv", sep=";", index=False)

print("\nFiles written to ./results/:")
print(" - rf_nova_eval_metrics_train_test_core.csv")
print(" - rf_nova_phase_coverage_stats.csv")
print(" - data_def_train_with_rf_nova_preds.csv")
print(" - data_def_test_with_rf_nova_preds.csv")

/var/folders/p2/b69xrmz146zc98k0yh6ylk1w0000gp/T/ipykernel_17851/1801042708.py:19: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv("data_def_train_folds.csv")
/var/folders/p2/b69xrmz146zc98k0yh6ylk1w0000gp/T/ipykernel_17851/1801042708.py:20: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  test  = pd.read_csv("data_def_test.csv")


Train shape: (206068, 24)
Test shape: (88315, 23)
Class mapping: {1: 0, 2: 1, 3: 2, 4: 3}

=== phase1_intrinsic – Using column 'labels_1_intrinsic' ===


/opt/anaconda3/lib/python3.12/site-packages/sklearn/preprocessing/_label.py:900: UserWarning: unknown class(es) ['ARTIFICIAL_FLAVOR', 'ENRICHED_WITH_OMEGA3', 'HIGH_IN_CARBOHYDRATES', 'HIGH_IN_OMEGA3_AND_OMEGA6', 'HIGH_IN_PROTEIN_AND_FIBRE', 'HIGH_IN_VITAMIN_K', 'KETOGENIC', 'LOW_FODMAP', 'MINIMALLY_PROCESSED', 'NATURAL_SUGARS_FROM_OATS', 'NATURAL_SUGARS_UNALTERED', 'NOT_FROZEN', 'NO_ADDED_ARTIFICIAL_SWEETENERS', 'NO_ADDITIVES_PRESERVATIVES_COLORANTS', 'NO_ANHYDRIDES', 'NO_AROMA_COLORANT_PRESERVATIVE', 'NO_ARTIFICIAL_DYES', 'NO_ARTIFICIAL_E_NUMBERS', 'NO_ARTIFICIAL_PRESERVATIVES_FLAVOURS_DYES', 'NO_CARRAGEENAN', 'NO_CHEMICAL_INGREDIENTS', 'NO_CHEMICAL_TREATMENT', 'NO_CLEANING_AGENTS', 'NO_COLORANT_AROMA_PRESERVATIVE', 'NO_DYES_PRESERVATIVES_HYDROGENATED_FATS', 'NO_EXTRACTS', 'NO_FRUIT', 'NO_GARLIC_ONION', 'NO_GLUTEN_COLORANT_PRESERVATIVE', 'NO_HIGH_PRESSURE_PROCESSING', 'NO_HONEY', 'NO_LARD', 'NO_METAL_MOLD', 'NO_NUT_TRACES', 'NO_PALM_OIL_DERIVATIVES', 'NO_PHOSPHORIC_ACID', 'NO_POTATO_F

Encoded label matrix shape (train): (206068, 359)
Encoded label matrix shape (test):  (88315, 359)
Coverage – phase1_intrinsic:
  Train:     80574/206068 (39.101% with ≥1 label in this phase)
  Test:      34289/88315 (38.826% with ≥1 label in this phase)
  Test-Core: 4875/4876 (99.979% with ≥1 label in this phase)
Fitted Random Forest for phase1_intrinsic in 5.60 s

Metrics – phase1_intrinsic
Train-Overall: n=206068, Acc=0.2792, BalAcc=0.3808, F1_macro=0.2537
Test-Overall: n=88315, Acc=0.2694, BalAcc=0.3737, F1_macro=0.2435
Test-Core: n=4876, Acc=0.6339, BalAcc=0.5735, F1_macro=0.4799
Train-PhaseCoverage: n=80574, Acc=0.6236, BalAcc=0.4847, F1_macro=0.4653
Test-PhaseCoverage: n=34289, Acc=0.6020, BalAcc=0.4611, F1_macro=0.4416

=== phase2_intr+extr – Using column 'labels_1_2_intr_extr' ===


/opt/anaconda3/lib/python3.12/site-packages/sklearn/preprocessing/_label.py:900: UserWarning: unknown class(es) ['ARTIFICIAL_FLAVOR', 'CERT_BIODYNAMIC', 'COASTAL_FISHING', 'DLG_BRONZE_MEDAL', 'ENRICHED_WITH_OMEGA3', 'EU_ORGANIC_CERT_FB', 'EU_ORGANIC_CERT_MY', 'GLUTEN_FREE_VEGAN', 'GLUTEN_LACTOSE_SULFITE_FREE', 'HAZELNUT_FREE', 'HIGH_IN_CARBOHYDRATES', 'HIGH_IN_OMEGA3_AND_OMEGA6', 'HIGH_IN_PROTEIN_AND_FIBRE', 'HIGH_IN_VITAMIN_K', 'INVALID_OR_ERROR', 'ISO_50001', 'KETOGENIC', 'LACTOSE_DAIRY_FREE', 'LOW_FODMAP', 'MAY_CONTAIN_SULPHITES', 'MILK_DERIVATIVE_FREE', 'MINIMALLY_PROCESSED', 'NATURAL_SUGARS_FROM_OATS', 'NATURAL_SUGARS_UNALTERED', 'NOT_FROZEN', 'NO_ADDED_ARTIFICIAL_SWEETENERS', 'NO_ADDITIVES_PRESERVATIVES_COLORANTS', 'NO_ANHYDRIDES', 'NO_ANIMAL_INGREDIENTS', 'NO_AROMA_COLORANT_PRESERVATIVE', 'NO_ARTIFICIAL_DYES', 'NO_ARTIFICIAL_E_NUMBERS', 'NO_ARTIFICIAL_PRESERVATIVES_FLAVOURS_DYES', 'NO_CARRAGEENAN', 'NO_CHEMICAL_INGREDIENTS', 'NO_CHEMICAL_TREATMENT', 'NO_CLEANING_AGENTS', 'NO_COL

Encoded label matrix shape (train): (206068, 808)
Encoded label matrix shape (test):  (88315, 808)
Coverage – phase2_intr+extr:
  Train:     185867/206068 (90.197% with ≥1 label in this phase)
  Test:      79571/88315 (90.099% with ≥1 label in this phase)
  Test-Core: 4876/4876 (100.000% with ≥1 label in this phase)
Fitted Random Forest for phase2_intr+extr in 60.17 s

Metrics – phase2_intr+extr
Train-Overall: n=206068, Acc=0.5349, BalAcc=0.5514, F1_macro=0.4565
Test-Overall: n=88315, Acc=0.4899, BalAcc=0.4966, F1_macro=0.4081
Test-Core: n=4876, Acc=0.7258, BalAcc=0.6464, F1_macro=0.5637
Train-PhaseCoverage: n=185867, Acc=0.5871, BalAcc=0.5792, F1_macro=0.4904
Test-PhaseCoverage: n=79571, Acc=0.5373, BalAcc=0.5177, F1_macro=0.4372

=== phase3_all_labels – Using column 'labels_string' ===


/opt/anaconda3/lib/python3.12/site-packages/sklearn/preprocessing/_label.py:900: UserWarning: unknown class(es) ['ARTIFICIAL_FLAVOR', 'CARREFOUR_CLASSIC', 'CERT_BIODYNAMIC', 'COASTAL_FISHING', 'DLG_BRONZE_MEDAL', 'ENRICHED_WITH_OMEGA3', 'EU_ORGANIC_CERT_FB', 'EU_ORGANIC_CERT_MY', 'GLUTEN_FREE_VEGAN', 'GLUTEN_LACTOSE_SULFITE_FREE', 'GOLD_MEDAL_GENERIC', 'GOLD_MEDAL_MILK_GRADE_A', 'HAZELNUT_FREE', 'HIGH_IN_CARBOHYDRATES', 'HIGH_IN_OMEGA3_AND_OMEGA6', 'HIGH_IN_PROTEIN_AND_FIBRE', 'HIGH_IN_VITAMIN_K', 'INVALID_OR_ERROR', 'ISO_50001', 'KETOGENIC', 'LACTOSE_DAIRY_FREE', 'LOW_FODMAP', 'MAY_CONTAIN_SULPHITES', 'MILK_DERIVATIVE_FREE', 'MINIMALLY_PROCESSED', 'NATURAL_SUGARS_FROM_OATS', 'NATURAL_SUGARS_UNALTERED', 'NOT_FROZEN', 'NO_ADDED_ARTIFICIAL_SWEETENERS', 'NO_ADDITIVES_PRESERVATIVES_COLORANTS', 'NO_ANHYDRIDES', 'NO_ANIMAL_INGREDIENTS', 'NO_AROMA_COLORANT_PRESERVATIVE', 'NO_ARTIFICIAL_DYES', 'NO_ARTIFICIAL_E_NUMBERS', 'NO_ARTIFICIAL_PRESERVATIVES_FLAVOURS_DYES', 'NO_CARRAGEENAN', 'NO_CHEMICA

Encoded label matrix shape (train): (206068, 870)
Encoded label matrix shape (test):  (88315, 870)
Coverage – phase3_all_labels:
  Train:     206068/206068 (100.000% with ≥1 label in this phase)
  Test:      88302/88315 (99.985% with ≥1 label in this phase)
  Test-Core: 4876/4876 (100.000% with ≥1 label in this phase)
Fitted Random Forest for phase3_all_labels in 81.49 s

Metrics – phase3_all_labels
Train-Overall: n=206068, Acc=0.5670, BalAcc=0.5758, F1_macro=0.4797
Test-Overall: n=88315, Acc=0.5090, BalAcc=0.5033, F1_macro=0.4174
Test-Core: n=4876, Acc=0.7406, BalAcc=0.6335, F1_macro=0.5899
Train-PhaseCoverage: n=206068, Acc=0.5670, BalAcc=0.5758, F1_macro=0.4797
Test-PhaseCoverage: n=88302, Acc=0.5090, BalAcc=0.5034, F1_macro=0.4175

=== All Metrics (Train-Overall, Test-Overall, Test-Core, PhaseCoverage) ===
                Phase              Dataset  n_samples  Accuracy  \
0    phase1_intrinsic        Train-Overall     206068  0.279160   
1    phase1_intrinsic         Test-Overall  